<h1 align="center"><font color="yellow">DeepLearning.AI</font></h1>

<font color="yellow">Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

# <font color="red">Overview of embeddings-based retrieval</font>

In [28]:
# from helper_utils import word_wrap
from pypdf import PdfReader

reader = PdfReader("spacy_teste.pdf") # "microsoft_annual_report_2022.pdf"
# https://s172-31-7-241p51602.lab-aws-production.deeplearning.ai/files/microsoft_annual_report_2022.pdf
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

pdf_texts


['CHATGPT: Granular ity clust ering\nQ: Que significa a granular idade de um clust ering\nA granularidade em um clustering refere-se ao nív el de detalhament o da divisão  \ndos dados em gr upos ou clust ers. \nPor exemplo, em um conjunto de dados de vendas de produtos em uma loja,  \nA escolha da granularidade depende do objetivo do clustering e da interpretação  \ndos resultados. Em alguns casos,  uma granular idade alta pode ajudar a  \nidentificar p adrões e caract erísticas específicas dos dados , enquanto em outros  \ncasos, uma granular idade b aixa pode ser suficient e para identificar t endências  \ngerais e insights.\nQ: Podes fazer ex emplos do conceit o\nClaro, aqui estão alguns exemplos para ilustrar o conceito de granularidade em  \nclustering:\nExemplo 1: Clust ering de compras online\nImagine que você esteja trabalhando em uma empresa de comércio eletrônico e  \nqueira agrupar os clientes com base em suas compras.  \nExemplo 2: Clust ering de dados de saúde\nSuponha que

In [29]:
len(pdf_texts)

5

In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter


character_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""],
                                                    chunk_size=650,
                                                    chunk_overlap=150
                                                   )

character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

#print(word_wrap(character_split_texts[10]))
print(f"\nTotal chunks: {len(character_split_texts)}")



Total chunks: 22


In [31]:
token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

#print(word_wrap(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")



Total chunks: 22


In [32]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()



print(embedding_function([token_split_texts[10]]))


[[0.050475094467401505, -0.02528349496424198, 0.042065612971782684, -0.02899896539747715, -0.09450729191303253, 0.004140762612223625, 0.049940671771764755, 0.01359529234468937, 0.032259680330753326, 0.019457556307315826, 0.07194534689188004, -0.03042110614478588, 0.04953896626830101, -0.03161582723259926, -0.053436748683452606, 0.014779279939830303, 0.0020094281062483788, -0.012086725793778896, -0.07398997247219086, -0.00022097196779213846, 0.09365537762641907, -0.06717002391815186, -0.006941597908735275, 0.08620414137840271, -0.07440692186355591, 0.051336634904146194, -0.038922905921936035, 0.022895514965057373, -0.03195254132151604, -0.02764248102903366, -0.028893396258354187, 0.06789273023605347, 0.12521414458751678, 0.022858591750264168, 0.018968665972352028, -0.011100003495812416, -0.009443390183150768, 0.03208250552415848, -0.03504585847258568, 0.11980273574590683, -0.09036161005496979, -0.047133706510066986, -0.08453940600156784, -0.050611913204193115, -0.01722913794219494, -0.0

In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("spacy_eddy", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

In [40]:
query = "O que é granularidade?" #"O que é entropia?" #"O que é a biblioteca spacy?" #"O que é granularidade?" #"O que é entropia?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    #print(word_wrap(document))
    print(document)
    print('\n')

chatgpt : granular ity clust ering q : que significa a granular idade de um clust ering a granularidade em um clustering refere - se ao niv el de detalhament o da divisao dos dados em gr upos ou clust ers. por exemplo, em um conjunto de dados de vendas de produtos em uma loja, a escolha da granularidade depende do objetivo do clustering e da interpretacao dos resultados. em alguns casos, uma granular idade alta pode ajudar a identificar p adroes e caract eristicas especificas dos dados, enquanto em outros casos, uma granular idade b aixa pode ser suficient e para identificar t endencias gerais e insights.


em geral, o objetivo do problema de classificacao e encontrar a granularidade mais apropriada que permita uma separacao eficiente dos dados de ambas as classes. para isso, e necessario utilizar tecnicas de classificacao, como a regressao logistica, arvores de decisao, svm, redes neurais, entre outras, para encontrar a fronteira de decisao otima entre as classes. a granularidade idea

In [41]:
# Substitua sua chave de API OpenAI:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.environ['OPENAI_API_KEY']


openai_client = OpenAI()

In [42]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "Você é um assistente especializado e útil e você só fala português. Seus usuários estão fazendo perguntas sobre as informações contidas em um documento de pesquisa."
            "Será mostrada a pergunta do usuário e as informações relevantes do documento de pesquisa. Responda à pergunta do usuário usando apenas essas informações."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content



In [43]:
output = rag(query=query, retrieved_documents=retrieved_documents)

#print(word_wrap(output))
print(output)


Granularidade refere-se ao nível de detalhamento da divisão dos dados em grupos ou clusters. Em um conjunto de dados de vendas de produtos em uma loja, a granularidade depende do objetivo do clustering e da interpretação dos resultados. Em alguns casos, uma granularidade alta pode ajudar a identificar padrões e características específicas dos dados, enquanto em outros casos, uma granularidade baixa pode ser suficiente para identificar tendências gerais e insights. A escolha da granularidade depende do contexto e do tipo de dados que estão sendo clusterizados.
